## Q16.1

这里定义一个二叉树来进行计算。通过递归可以计算出0时刻期权的价格为56。

In [1]:
class Node:
    """A binary tree node.

    Args:
        stock: stock price at this node
        u: up rate
        d: down rate
        er: risk-free rate of interest
    """
    def __init__(self, stock, u, d, er):
        self.stock  = stock
        self.u      = u
        self.d      = d
        self.er     = er

        self.parent = None      ## parent node
        self.left   = None      ## left child node (the up branch)
        self.right  = None      ## right child node (the down branch)
    
    def is_leaf(self):
        return self.left == None and self.right == None
    
    @property
    def q(self):
        return (self.er - self.d) / (self.u - self.d)
    
    @property
    def up_branch(self):
        if self.left == None:
            self.left = Node(self.stock * self.u, self.u, self.d, self.er)
            self.left.parent = self
        
        return self.left
    
    @property
    def down_branch(self):
        if self.right == None:
            self.right = Node(self.stock * self.d, self.u, self.d, self.er)
            self.right.parent = self
        
        return self.right

In [2]:
## information at t = 0
stock = 100
u = 2.0
d = 0.5
er = 1.25

## initialize the binary tree structure
## t = 0
root = Node(stock, u, d, er)

## t = 1
t1 = [root.up_branch, root.down_branch]

## t = 2
t2 = []
for node in t1:
    t2.append(node.up_branch)
    t2.append(node.down_branch)

## option function
def Option(node):
    if node.is_leaf():
        s2 = node.stock
        s1 = node.parent.stock
        s0 = node.parent.parent.stock

        return max(s2 - min([s0, s1, s2]), 0)
    
    q  = node.q
    er = node.er

    return (q * Option(node.up_branch) + (1-q) * Option(node.down_branch)) / er


print(f"The option price at t=0 is {Option(root):.2f}.")

The option price at t=0 is 56.00.
